# Data


In [ ]:
import pandas as pd

In [ ]:
#connect to G-Drive
from google.colab import drive
mount='/content/gdrive'
drive.mount(mount)

Mounted at /content/gdrive


In [ ]:
%cd "/content/gdrive/MyDrive/Colab Notebooks/Topic Modelling"

/content/gdrive/MyDrive/Colab Notebooks/Topic Modelling


In [ ]:
result = pd.read_csv("topic_full_strict.csv", sep=',')

In [ ]:
result.head()

,Unnamed: 0,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,process_id
0,0,Φυσική ταυτοποίηση πολίτη στο ΚΕΠ,43,43_ταυτοποίηση πολίτη_φυσική ταυτοποίηση_φυσικ...,"['ταυτοποίηση πολίτη', 'φυσική ταυτοποίηση', '...","['Φυσική ταυτοποίηση πολίτη', 'Φυσική ταυτοποί...",ταυτοποίηση πολίτη - φυσική ταυτοποίηση - φυσι...,0.162237,False,439993
1,1,Υποβολή αίτησης στο Πληροφοριακό σύστημα,26,26_ηλεκτρονική_αριθμού_αριθμού πρωτοκόλλου_ψηφ...,"['ηλεκτρονική', 'αριθμού', 'αριθμού πρωτοκόλλο...",['Παραλαβή αιτήματος και χορήγηση αριθμού πρωτ...,ηλεκτρονική - αριθμού - αριθμού πρωτοκόλλου - ...,0.992964,False,439993
2,2,Εκτύπωση αποδεικτικού φορολογικής ενημερότητας,0,0_ποσού_πληρωμής_πληρωμή_χρηματικού,"['ποσού', 'πληρωμής', 'πληρωμή', 'χρηματικού',...","['Αποστολή Αποδεικτικών Πληρωμής', 'Αποστολή Α...",ποσού - πληρωμής - πληρωμή - χρηματικού - οικο...,1.000000,False,439993
3,3,Εκκίνηση διαδικασίας,3,3_διαδικασίας εκκίνηση_διαδικασίας εκκίνηση δι...,"['διαδικασίας εκκίνηση', 'διαδικασίας εκκίνηση...","['Εκκίνηση διαδικασίας', 'Εκκίνηση διαδικασίας...",διαδικασίας εκκίνηση - διαδικασίας εκκίνηση δι...,0.789207,True,937099
4,4,Παραλαβή αιτήματος για διακίνηση χοιροειδών,2,2_άδειας_εμπορίας_κατηγορίας_άδειας εμπορίας,"['άδειας', 'εμπορίας', 'κατηγορίας', 'άδειας ε...","['Χορήγηση Άδειας Εμπορίας Κατηγορίας Γ', 'Χορ...",άδειας - εμπορίας - κατηγορίας - άδειας εμπορί...,1.000000,False,937099


In [ ]:
len(result)

19931

# **Semantic Similarity**



## FAISS optimized


In [ ]:
!pip install faiss-cpu
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 70.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=b482bb0ab98086a4412b04c0035237fa31481a91fc2277ceaf1396114f223e9a
  Stored in directory

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
import numpy as np
import faiss

def get_sentence_embeddings(sentences, model):
    sentence_embeddings = model.encode(sentences, convert_to_tensor=True)
    return sentence_embeddings.cpu()  # Move the embeddings to the CPU

def build_faiss_index(embeddings):
    index = faiss.IndexFlatIP(embeddings.shape[1])
    index.add(embeddings)
    return index

def find_most_similar_sentence(query_embedding, index, embeddings, similarity_threshold, kept_sentences):
    similarity_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    top_similarities = similarity_scores[similarity_scores >= similarity_threshold]
    if len(top_similarities) > 0:
        max_similarity = top_similarities.max().item()
        max_similarity_index = np.argmax(similarity_scores)
        if max_similarity >= similarity_threshold:
            return kept_sentences[max_similarity_index], max_similarity
    return '', 0.0

def filter_similar_sentences(dataframe, similarity_threshold=0.95):
    model_name = 'paraphrase-multilingual-mpnet-base-v2'
    model = SentenceTransformer(model_name)

    # Create new columns to store the most similar sentence and similarity measure
    dataframe['Most_Similar_Sentence'] = ''
    dataframe['Similarity_Measure'] = 0.0

    # Initialize the kept_sentences list and embeddings
    kept_sentences = []
    kept_embeddings = []
    kept_index = None

    # Iterate over the rows in the dataframe
    for i, row in dataframe.iterrows():
        curr_sentence = row['Document']
        if isinstance(curr_sentence, str):  # Check if the sentence is a valid string
            curr_embedding = get_sentence_embeddings([curr_sentence], model)[0]  # Pass the model as an argument
            most_similar_sentence = ''
            max_similarity = 0.0

            # Compare with the previously kept sentences using FAISS index
            if kept_index is not None:
                most_similar_sentence, max_similarity = find_most_similar_sentence(
                    curr_embedding, kept_index, kept_embeddings, similarity_threshold, kept_sentences
                )

            # If a similar sentence is found, update the most similar sentence and similarity measure columns
            if most_similar_sentence != '':
                dataframe.loc[i, 'Most_Similar_Sentence'] = most_similar_sentence
                dataframe.loc[i, 'Similarity_Measure'] = max_similarity
            else:
                # If no similar sentence is found, keep the original text instead of the embedding
                kept_sentences.append(curr_sentence)
                dataframe.loc[i, 'Most_Similar_Sentence'] = curr_sentence
                dataframe.loc[i, 'Similarity_Measure'] = 1.0  # Set similarity measure to 1.0 for the original sentence

            # Check if the current sentence is similar to the kept sentences
            if most_similar_sentence == '' or max_similarity < similarity_threshold:
                kept_embeddings.append(curr_embedding.cpu().numpy().tolist())  # Convert numpy array to Python list
                if kept_index is None:
                    kept_index = build_faiss_index(np.array([curr_embedding.cpu().numpy()]))
                else:
                    kept_index.add(np.array([curr_embedding.cpu().numpy()]))

        else:
            dataframe.loc[i, 'Most_Similar_Sentence'] = np.nan
            dataframe.loc[i, 'Similarity_Measure'] = np.nan

    return dataframe

### Small Example

In [ ]:
sentences = result.loc[result['Topic'] == 20]

filtered_df = filter_similar_sentences(sentences, similarity_threshold=0.85)

<ipython-input-8-9822ea397ad8>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Most_Similar_Sentence'] = ''
<ipython-input-8-9822ea397ad8>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Similarity_Measure'] = 0.0


In [ ]:
len(filtered_df)

204

In [ ]:
filtered_df[0:50]

,Unnamed: 0,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,process_id,Most_Similar_Sentence,Similarity_Measure
243,243,Ορθή / πλήρης αίτηση/δικαιολογητικά : Επιτόπιο...,20,20_έλεγχος αίτησης_παραλαβή έλεγχος αίτησης_πα...,"['έλεγχος αίτησης', 'παραλαβή έλεγχος αίτησης'...",['Παραλαβή και έλεγχος της αίτησης και λοιπών ...,έλεγχος αίτησης - παραλαβή έλεγχος αίτησης - π...,0.000000,False,962992,Ορθή / πλήρης αίτηση/δικαιολογητικά : Επιτόπιο...,1.000000
282,282,Έλεγχος αίτησης έναρξης εκμετάλλευσης,20,20_έλεγχος αίτησης_παραλαβή έλεγχος αίτησης_πα...,"['έλεγχος αίτησης', 'παραλαβή έλεγχος αίτησης'...",['Παραλαβή και έλεγχος της αίτησης και λοιπών ...,έλεγχος αίτησης - παραλαβή έλεγχος αίτησης - π...,0.000000,False,621184,Έλεγχος αίτησης έναρξης εκμετάλλευσης,1.000000
324,324,Έγκριση της Aίτησης κατόπιν ελέγχου των δικαιο...,20,20_έλεγχος αίτησης_παραλαβή έλεγχος αίτησης_πα...,"['έλεγχος αίτησης', 'παραλαβή έλεγχος αίτησης'...",['Παραλαβή και έλεγχος της αίτησης και λοιπών ...,έλεγχος αίτησης - παραλαβή έλεγχος αίτησης - π...,0.893740,False,542209,Έγκριση της Aίτησης κατόπιν ελέγχου των δικαιο...,1.000000
470,470,Έλεγχος περιεχομένου της αίτησης.,20,20_έλεγχος αίτησης_παραλαβή έλεγχος αίτησης_πα...,"['έλεγχος αίτησης', 'παραλαβή έλεγχος αίτησης'...",['Παραλαβή και έλεγχος της αίτησης και λοιπών ...,έλεγχος αίτησης - παραλαβή έλεγχος αίτησης - π...,0.000000,False,479753,Έλεγχος περιεχομένου της αίτησης.,1.000000
488,488,Επεξεργασία των ηλεκτρονικών αιτήσεων και έλεγ...,20,20_έλεγχος αίτησης_παραλαβή έλεγχος αίτησης_πα...,"['έλεγχος αίτησης', 'παραλαβή έλεγχος αίτησης'...",['Παραλαβή και έλεγχος της αίτησης και λοιπών ...,έλεγχος αίτησης - παραλαβή έλεγχος αίτησης - π...,1.000000,False,201794,Επεξεργασία των ηλεκτρονικών αιτήσεων και έλεγ...,1.000000
534,534,Επεξεργασία των ηλεκτρονικών αιτήσεων και έλεγ...,20,20_έλεγχος αίτησης_παραλαβή έλεγχος αίτησης_πα...,"['έλεγχος αίτησης', 'παραλαβή έλεγχος αίτησης'...",['Παραλαβή και έλεγχος της αίτησης και λοιπών ...,έλεγχος αίτησης - παραλαβή έλεγχος αίτησης - π...,0.970585,False,227405,Επεξεργασία των ηλεκτρονικών αιτήσεων και έλεγ...,1.000000
634,634,Έλεγχος αίτησης και υποβληθέντων δικαιολογητικών,20,20_έλεγχος αίτησης_παραλαβή έλεγχος αίτησης_πα...,"['έλεγχος αίτησης', 'παραλαβή έλεγχος αίτησης'...",['Παραλαβή και έλεγχος της αίτησης και λοιπών ...,έλεγχος αίτησης - παραλαβή έλεγχος αίτησης - π...,0.000000,False,188918,Έγκριση της Aίτησης κατόπιν ελέγχου των δικαιο...,0.937106
1177,1177,Έλεγχος της αίτησης έγκρισης τύπου Μ.Ε.,20,20_έλεγχος αίτησης_παραλαβή έλεγχος αίτησης_πα...,"['έλεγχος αίτησης', 'παραλαβή έλεγχος αίτησης'...",['Παραλαβή και έλεγχος της αίτησης και λοιπών ...,έλεγχος αίτησης - παραλαβή έλεγχος αίτησης - π...,0.000000,False,989656,Έλεγχος της αίτησης έγκρισης τύπου Μ.Ε.,1.000000
1472,1472,Χρέωση και έλεγχος αιτήματος,20,20_έλεγχος αίτησης_παραλαβή έλεγχος αίτησης_πα...,"['έλεγχος αίτησης', 'παραλαβή έλεγχος αίτησης'...",['Παραλαβή και έλεγχος της αίτησης και λοιπών ...,έλεγχος αίτησης - παραλαβή έλεγχος αίτησης - π...,0.000000,False,812816,Χρέωση και έλεγχος αιτήματος,1.000000
1539,1539,Έλεγχος ταυτοπροσωπίας και συνδρομής νόμιμων π...,20,20_έλεγχος αίτησης_παραλαβή έλεγχος αίτησης_πα...,"['έλεγχος αίτησης', 'παραλαβή έλεγχος αίτησης'...",['Παραλαβή και έλεγχος της αίτησης και λοιπών ...,έλεγχος αίτησης - παραλαβή έλεγχος αίτησης - π...,0.000000,False,144400,Έλεγχος ταυτοπροσωπίας και συνδρομής νόμιμων π...,1.000000


## Entire List

In [ ]:
topics_num = result["Topic"].nunique()-1
print(topics_num)

139


In [ ]:
final_df = pd.DataFrame()

for j in range(-1, topics_num):
#for j in range(300, 400):

  sentences = result.loc[result['Topic'] == j]
  filtered_df = filter_similar_sentences(sentences, similarity_threshold=0.85)

  final_df = pd.concat([final_df, filtered_df], ignore_index=True)
  #print(j, final_steps)
final_df.to_csv('final_document_full_paraphrase-multilingual-mpnet-base-v2-strict).csv', index=False)

<ipython-input-8-9822ea397ad8>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Most_Similar_Sentence'] = ''
<ipython-input-8-9822ea397ad8>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Similarity_Measure'] = 0.0
<ipython-input-8-9822ea397ad8>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [ ]:
final_df.head(50)

,Unnamed: 0,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,process_id,Most_Similar_Sentence,Similarity_Measure
0,38,Αναγνώριση πλασματικού χρόνου,-1,-1_εγγραφή_επιτροπή_προς_μέσω,"['εγγραφή', 'επιτροπή', 'προς', 'μέσω', 'υπηρε...",['Δημιουργία ψηφιακού ακριβούς αντιγράφου Δελτ...,εγγραφή - επιτροπή - προς - μέσω - υπηρεσίες -...,0.0,False,214376,Αναγνώριση πλασματικού χρόνου,1.000000
1,48,Εξέταση μαθημάτων για πτυχία Ιατρικής ή Οδοντι...,-1,-1_εγγραφή_επιτροπή_προς_μέσω,"['εγγραφή', 'επιτροπή', 'προς', 'μέσω', 'υπηρε...",['Δημιουργία ψηφιακού ακριβούς αντιγράφου Δελτ...,εγγραφή - επιτροπή - προς - μέσω - υπηρεσίες -...,0.0,False,814359,Εξέταση μαθημάτων για πτυχία Ιατρικής ή Οδοντι...,1.000000
2,55,Μη εγγραφή στο μητρώο,-1,-1_εγγραφή_επιτροπή_προς_μέσω,"['εγγραφή', 'επιτροπή', 'προς', 'μέσω', 'υπηρε...",['Δημιουργία ψηφιακού ακριβούς αντιγράφου Δελτ...,εγγραφή - επιτροπή - προς - μέσω - υπηρεσίες -...,0.0,False,257811,Μη εγγραφή στο μητρώο,1.000000
3,57,Έκδοση καρτέλας αμπελουργικού μητρώου,-1,-1_εγγραφή_επιτροπή_προς_μέσω,"['εγγραφή', 'επιτροπή', 'προς', 'μέσω', 'υπηρε...",['Δημιουργία ψηφιακού ακριβούς αντιγράφου Δελτ...,εγγραφή - επιτροπή - προς - μέσω - υπηρεσίες -...,0.0,False,257811,Έκδοση καρτέλας αμπελουργικού μητρώου,1.000000
4,66,Επιλογή έκδοσης Βεβαίωσης ανεργίας για παροχή ...,-1,-1_εγγραφή_επιτροπή_προς_μέσω,"['εγγραφή', 'επιτροπή', 'προς', 'μέσω', 'υπηρε...",['Δημιουργία ψηφιακού ακριβούς αντιγράφου Δελτ...,εγγραφή - επιτροπή - προς - μέσω - υπηρεσίες -...,0.0,False,374859,Επιλογή έκδοσης Βεβαίωσης ανεργίας για παροχή ...,1.000000
5,79,Επιτόπιος έλεγχος για τη συλλογή και την ταυτο...,-1,-1_εγγραφή_επιτροπή_προς_μέσω,"['εγγραφή', 'επιτροπή', 'προς', 'μέσω', 'υπηρε...",['Δημιουργία ψηφιακού ακριβούς αντιγράφου Δελτ...,εγγραφή - επιτροπή - προς - μέσω - υπηρεσίες -...,0.0,False,161114,Επιτόπιος έλεγχος για τη συλλογή και την ταυτο...,1.000000
6,80,Επιτόπιος Ελεγχος για την προσωρινή αποθήκευση...,-1,-1_εγγραφή_επιτροπή_προς_μέσω,"['εγγραφή', 'επιτροπή', 'προς', 'μέσω', 'υπηρε...",['Δημιουργία ψηφιακού ακριβούς αντιγράφου Δελτ...,εγγραφή - επιτροπή - προς - μέσω - υπηρεσίες -...,0.0,False,161114,Επιτόπιος Ελεγχος για την προσωρινή αποθήκευση...,1.000000
7,81,Επιτόπιος 'Ελεγχος για την τήρηση των κανόνων...,-1,-1_εγγραφή_επιτροπή_προς_μέσω,"['εγγραφή', 'επιτροπή', 'προς', 'μέσω', 'υπηρε...",['Δημιουργία ψηφιακού ακριβούς αντιγράφου Δελτ...,εγγραφή - επιτροπή - προς - μέσω - υπηρεσίες -...,0.0,False,161114,Επιτόπιος 'Ελεγχος για την τήρηση των κανόνων...,1.000000
8,85,Επιτυχία εισόδου στο σύστημα,-1,-1_εγγραφή_επιτροπή_προς_μέσω,"['εγγραφή', 'επιτροπή', 'προς', 'μέσω', 'υπηρε...",['Δημιουργία ψηφιακού ακριβούς αντιγράφου Δελτ...,εγγραφή - επιτροπή - προς - μέσω - υπηρεσίες -...,0.0,False,732020,Επιτυχία εισόδου στο σύστημα,1.000000
9,86,Αποτυχία εισόδου στο σύστημα,-1,-1_εγγραφή_επιτροπή_προς_μέσω,"['εγγραφή', 'επιτροπή', 'προς', 'μέσω', 'υπηρε...",['Δημιουργία ψηφιακού ακριβούς αντιγράφου Δελτ...,εγγραφή - επιτροπή - προς - μέσω - υπηρεσίες -...,0.0,False,732020,Αποτυχία εισόδου στο σύστημα,1.000000


In [ ]:
print(final_df.Most_Similar_Sentence.unique())
print(final_df.Most_Similar_Sentence.nunique())

['Αναγνώριση πλασματικού χρόνου'
 'Εξέταση μαθημάτων για πτυχία Ιατρικής ή Οδοντιατρικής κρατών εκτός Ε.Ε.'
 'Μη εγγραφή στο μητρώο' ...
 'Παράδοση του πιστοποιητικού στον πολίτη ή αποστολή ταχυδρομικώς'
 'Καταχώρηση στοιχέιων σε Μητρώο'
 'Αποστολή πιστοποιητικών από τους δήμους στο Υπουργείο Εσωτερικών και καταχώρησή τους στους εκλογικούς καταλόγους']
5899


In [ ]:
sentences = final_df.loc[final_df['Topic'] == 30]

In [ ]:
sentences.sort_values('Similarity_Measure', ascending=True)

,Unnamed: 0,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,process_id,Most_Similar_Sentence,Similarity_Measure
10725,14297,Λήψη φακέλου,30,30_στοιχείων_καταχώριση_στοιχείων μητρώο_αρχείου,"['στοιχείων', 'καταχώριση', 'στοιχείων μητρώο'...","['1ο μέρος: Καταχώριση στοιχείων στο μητρώο', ...",στοιχείων - καταχώριση - στοιχείων μητρώο - αρ...,0.930167,False,356162,Αρχειοθέτηση φακέλου.,0.850352
10638,1384,Καταχώρηση απογραφικού – αναλώσιμου υλικού από...,30,30_στοιχείων_καταχώριση_στοιχείων μητρώο_αρχείου,"['στοιχείων', 'καταχώριση', 'στοιχείων μητρώο'...","['1ο μέρος: Καταχώριση στοιχείων στο μητρώο', ...",στοιχείων - καταχώριση - στοιχείων μητρώο - αρ...,0.000000,False,673065,Προσωρινή Εισαγωγή απογραφικού – αναλώσιμου υλ...,0.851677
10717,11827,Επισύναψη αρχείων,30,30_στοιχείων_καταχώριση_στοιχείων μητρώο_αρχείου,"['στοιχείων', 'καταχώριση', 'στοιχείων μητρώο'...","['1ο μέρος: Καταχώριση στοιχείων στο μητρώο', ...",στοιχείων - καταχώριση - στοιχείων μητρώο - αρ...,0.941680,False,542124,Σύνταξη εγγράφου με τις αποθηκευμένες καταχωρί...,0.852929
10766,18830,Σύνταξη εγγράφου για προσκόμιση τυχόν συμπληρω...,30,30_στοιχείων_καταχώριση_στοιχείων μητρώο_αρχείου,"['στοιχείων', 'καταχώριση', 'στοιχείων μητρώο'...","['1ο μέρος: Καταχώριση στοιχείων στο μητρώο', ...",στοιχείων - καταχώριση - στοιχείων μητρώο - αρ...,0.000000,False,376266,Εισαγωγή συμπληρωματικών στοιχείων,0.854343
10760,18251,Θεώρηση Τοπογραφικού Διαγράμματος,30,30_στοιχείων_καταχώριση_στοιχείων μητρώο_αρχείου,"['στοιχείων', 'καταχώριση', 'στοιχείων μητρώο'...","['1ο μέρος: Καταχώριση στοιχείων στο μητρώο', ...",στοιχείων - καταχώριση - στοιχείων μητρώο - αρ...,0.000000,False,282682,Σύνταξη τοπογραφικού διαγράμματος,0.855428
...,...,...,...,...,...,...,...,...,...,...,...,...
10671,3658,Καταχώρηση στοιχείων στο μητρώο,30,30_στοιχείων_καταχώριση_στοιχείων μητρώο_αρχείου,"['στοιχείων', 'καταχώριση', 'στοιχείων μητρώο'...","['1ο μέρος: Καταχώριση στοιχείων στο μητρώο', ...",στοιχείων - καταχώριση - στοιχείων μητρώο - αρ...,0.981034,False,394059,Καταχώρηση στοιχείων στο μητρώο,1.000000
10626,272,Καταχώριση στοιχείων στο ΟΠΣ Κτηνιατρικής,30,30_στοιχείων_καταχώριση_στοιχείων μητρώο_αρχείου,"['στοιχείων', 'καταχώριση', 'στοιχείων μητρώο'...","['1ο μέρος: Καταχώριση στοιχείων στο μητρώο', ...",στοιχείων - καταχώριση - στοιχείων μητρώο - αρ...,0.000000,False,779845,Καταχώριση στοιχείων στο ΟΠΣ Κτηνιατρικής,1.000000
10659,3373,1ο μέρος: Καταχώριση στοιχείων στο μητρώο,30,30_στοιχείων_καταχώριση_στοιχείων μητρώο_αρχείου,"['στοιχείων', 'καταχώριση', 'στοιχείων μητρώο'...","['1ο μέρος: Καταχώριση στοιχείων στο μητρώο', ...",στοιχείων - καταχώριση - στοιχείων μητρώο - αρ...,0.966052,True,763859,1ο μέρος: Καταχώριση στοιχείων στο μητρώο,1.000000
10777,19739,Καταχώρηση αριθμού μετρητή και αρ. μητρώου ΕΥΔΑΠ,30,30_στοιχείων_καταχώριση_στοιχείων μητρώο_αρχείου,"['στοιχείων', 'καταχώριση', 'στοιχείων μητρώο'...","['1ο μέρος: Καταχώριση στοιχείων στο μητρώο', ...",στοιχείων - καταχώριση - στοιχείων μητρώο - αρ...,0.000000,False,788281,Καταχώρηση αριθμού μετρητή και αρ. μητρώου ΕΥΔΑΠ,1.000000
